In [11]:
from pymatgen.ext.matproj import MPRester
#from mp_api import MPRester
with MPRester("brO6w6AHZNfzGnTWuqOM") as m:
    data = m.query(criteria={"deprecated": False}, properties=['structure','formation_energy_per_atom','pretty_formula'])

100%|█████████████████████████████████████████████████████████████████████████| 126335/126335 [15:46<00:00, 133.46it/s]


In [ ]:
with MPRester("brO6w6AHZNfzGnTWuqOM") as m:
    data = m.query()

In [3]:
with MPRester("brO6w6AHZNfzGnTWuqOM") as m:
    data2 = m.query(criteria={"deprecated": True}, properties=['structure','e_above_hull','pretty_formula'])

  2%|▏         | 3000/126335 [00:15<10:48, 190.12it/s]

KeyboardInterrupt: 

In [13]:
print(data[4])

{'structure': Structure Summary
Lattice
    abc : 10.263495690955983 10.263495690955983 10.263495690955983
 angles : 109.47122063449069 109.47122063449069 109.47122063449069
 volume : 832.2695819600563
      A : -5.925632 5.925632 5.925632
      B : 5.925632 -5.925632 5.925632
      C : 5.925632 5.925632 -5.925632
PeriodicSite: Li (4.4442, 5.9256, 2.9628) [0.7500, 0.6250, 0.8750]
PeriodicSite: Li (7.4070, 0.0000, 2.9628) [0.2500, 0.8750, 0.6250]
PeriodicSite: Li (2.9628, 7.4070, 0.0000) [0.6250, 0.2500, 0.8750]
PeriodicSite: Li (2.9628, 4.4442, 5.9256) [0.8750, 0.7500, 0.6250]
PeriodicSite: Li (0.0000, 2.9628, 7.4070) [0.8750, 0.6250, 0.2500]
PeriodicSite: Li (5.9256, 2.9628, 4.4442) [0.6250, 0.8750, 0.7500]
PeriodicSite: Li (1.4814, 0.0000, 2.9628) [0.2500, 0.3750, 0.1250]
PeriodicSite: Li (-1.4814, 5.9256, 2.9628) [0.7500, 0.1250, 0.3750]
PeriodicSite: Li (2.9628, -1.4814, 5.9256) [0.3750, 0.7500, 0.1250]
PeriodicSite: Li (2.9628, 1.4814, 0.0000) [0.1250, 0.2500, 0.3750]
PeriodicSite

In [25]:
from scipy.sparse import identity
from scipy.sparse.linalg import eigs
def findMinEigenvalue(c11, c12, c13, c21, c22, c23, c31, c32, c33):
    matrix = [[c11, c12, c13],[c21, c22, c23],[c31, c32, c33]]
    matrix = np.array(matrix).reshape((3,3))
    return min(eigs(matrix, 3, sigma=0)[0].real)
def classificationStiffness(stiff):
    if(stiff < 0):
        return 0
    else:
        return 1
def classifyThermoStability(value):
    if(value <= 0.02):
        return 0
    else:
        return 2
import pandas as pd
df = pd.DataFrame()
from ase.db import connect
import pandas as pd
import os
from ase.db import connect
from pymatgen.core import Composition, Structure
from pymatgen.io.ase import AseAtomsAdaptor
from tqdm.autonotebook import tqdm
from scipy.sparse import identity
from scipy.sparse.linalg import eigs
import numpy as np
from aviary.cgcnn.utils import get_cgcnn_input
from aviary.wren.utils import count_wyks, get_aflow_label_spglib
from tqdm.notebook import tqdm
tqdm.pandas()

C:\Users\Matthew\AppData\Local\Temp/ipykernel_41048/1342673259.py:25: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [6]:
print(data[1000]['e_above_hull'])

0.021311409999999587


In [16]:
print(len(data))
print(type(data[100]['structure']))

126335
<class 'pymatgen.core.structure.Structure'>


In [19]:
materials = []
composition = []
classification = []
structs = []
for entry in data:
    try:
        classification.append(entry['formation_energy_per_atom'])
        structs.append(entry["structure"])
        materials.append(entry["pretty_formula"])
        composition.append(entry["pretty_formula"])
    except:
        pass

In [20]:
print(len(structs))
print(classification)

126335
[0.5371230525000001, 0.23428657927600013, -0.34923915662999994, -2.5171352157499998, -1.5430507707499992, -0.58613447375, -2.5801379335714287, -0.42690064952380957, -0.4332087812500003, -2.9545415595312496, -0.003951669444444435, 0.10974483000000035, 0.01860837267241333, -0.6845701809999994, -1.559203024238504, 0.14422689888888898, 0.10913994621212195, 0.0869975943939397, 0.2326130683333337, -0.27124356333333327, -0.7048155820833326, 1.992915160777778, 2.2668649525000006, -0.02147816583333295, -0.03655939722222238, -0.5037621522222224, -0.45708535333333256, 0.8402922183333335, -1.3529626376186663, 1.013371345, -0.8732702144444452, -2.4492045938938483, -1.1023754802777777, -0.13164252256410253, -0.37696954607142885, -1.7169678903333325, -2.233476780416667, -0.4513894565603446, -2.145904393798644, -2.3348591418060005, -0.42347168996484363, -1.0848195373362068, -2.0647546378755406, -1.6598241575422072, -1.0429282450862067, -2.7109913286335403, -2.8039701025139996, -0.14006853249999

In [26]:
import pandas
df = pandas.DataFrame()
df['material_id'] = materials
df['composition'] = composition
df['final_structure'] = structs
df['wyckoff'] = df['final_structure'].apply(get_aflow_label_spglib)
df['hform'] = classification

C:\Users\Matthew\.conda\envs\MachineLearning\lib\site-packages\pymatgen-2022.5.26-py3.8-win-amd64.egg\pymatgen\core\periodic_table.py:210: UserWarning: No electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
C:\Users\Matthew\.conda\envs\MachineLearning\lib\site-packages\pymatgen-2022.5.26-py3.8-win-amd64.egg\pymatgen\core\periodic_table.py:210: UserWarning: No electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
C:\Users\Matthew\.conda\envs\MachineLearning\lib\site-packages\pymatgen-2022.5.26-py3.8-win-amd64.egg\pymatgen\core\periodic_table.py:210: UserWarning: No electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(


In [ ]:
# Creating a dataframe with 75%
# values of original dataframe
part_75 = df.sample(frac = 0.80)
 
# Creating dataframe with
# rest of the 25% values
rest_part_25 = df.drop(part_75.index)

part_75.to_csv('C:\\Users\\Matthew\\Desktop\\HFormTraining.csv', index=False)

In [27]:
print(df)
df = df.drop(['final_structure'], axis=1)
df.to_csv('C:\\Users\\Matthew\\Desktop\\trainingTestHForm.csv', index=False)

          material_id    composition  \
0                 SiC            SiC   
1               HPbI3          HPbI3   
2                BrCl           BrCl   
3              EuFeO3         EuFeO3   
4       Li3Ni2(GeO4)3  Li3Ni2(GeO4)3   
...               ...            ...   
126330      Ca2CuIrO6      Ca2CuIrO6   
126331      Zn(SnO2)2      Zn(SnO2)2   
126332      ZnFeAs2O7      ZnFeAs2O7   
126333    CaSb2(PO5)2    CaSb2(PO5)2   
126334      Ca(CuO2)2      Ca(CuO2)2   

                                          final_structure  \
0               [[0. 0. 0.] Si, [2.0251 2.0251 2.0251] C]   
1       [[3.1221065 3.1221065 3.1221065] H, [0. 0. 0.]...   
2       [[0. 0. 0.] Br, [0.        0.        3.9053505...   
3       [[ 2.747224    2.72097164 -1.9501425 ] Eu, [5....   
4       [[4.444224 5.925632 2.962816] Li, [7.40704  0....   
...                                                   ...   
126330  [[3.15778709 8.03614231 5.77105139] Ca, [0.340...   
126331  [[ 0.         11.368298

In [5]:
index = 0
entries = []
values = []
structs = []
with MPRester("aU621ICtBbnsLKk9") as m:
    for j in data:
        i = j['material_id']
        values.append(0)
        structs.append(j['structure'])
        

In [6]:
import pandas as pd
df = pd.DataFrame()
df['final_structure'] = structs
df['dynamic_stability_phonons'] = values
from ase.db import connect
import pandas as pd
import os
from ase.db import connect
from pymatgen.core import Composition, Structure
from pymatgen.io.ase import AseAtomsAdaptor
from tqdm.autonotebook import tqdm
from scipy.sparse import identity
from scipy.sparse.linalg import eigs
import numpy as np
from aviary.cgcnn.utils import get_cgcnn_input
from aviary.wren.utils import count_wyks, get_aflow_label_spglib
from tqdm.notebook import tqdm
tqdm.pandas()

C:\Users\dylan\AppData\Local\Temp\ipykernel_14364\1548887149.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [7]:
name = []
for j in data:
    name.append(j['material_id'])
df['material_id'] = name
df['wyckoff'] = df['final_structure'].progress_apply(get_aflow_label_spglib)
df = df.drop(['final_structure'], axis=1)
df['composition'] = df['final_structure'].apply(lambda x: x.composition.reduced_formula)
df.drop('final_structure')

  0%|          | 0/5142 [00:00<?, ?it/s]

KeyError: "['final_structure'] not found in axis"

In [8]:
df = df.drop(['final_structure'], axis=1)

In [9]:
df.to_csv('C:\\Users\\dylan\\Desktop\\generatedData\\morePhonons2.csv', index=False)